In [1]:
from seleniumbase import Driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import mysql.connector


In [17]:
mydb = mysql.connector.connect(
    host = "media-studies-jobs.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user = "admin",
    password = "123456789",
    database = "indeed_jobs"
)

cursor = mydb.cursor()
cursor = mydb.cursor(buffered=True)


In [41]:
driver = Driver(uc = True)

# Setup explicit wait
wait = WebDriverWait(driver, 20)

In [6]:
import csv

In [39]:
import csv

# List of URLs to scrape
urls = [
    "https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d12",
    "https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=b8a5956f0918959c",
    "https://www.indeed.com/jobs?q=sustainable+agriculture&l=USA&vjk=b9b8bf61620566e1",
    "https://www.indeed.com/jobs?q=sustainability+consultant&l=USA&vjk=4709ed414e838017"
]

# Initialize list to store job data
jobs_data = []

max_pages = 1
for url in urls:
    page = 0  # Initialize page counter
    while page < max_pages:  # Limit the loop to 1 page per URL
        page_val = page * 12  # Assuming 10 is the pagination step
        full_url = f"{url}{page_val}"
        print("Fetching:", full_url)

        driver.get(full_url)

        # Wait for the job listings to be loaded
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
        except TimeoutException:
            print("Timed out waiting for page to load")
            continue  # Skip to the next iteration if the page doesn't load

        print(f"Page {page + 1} done.")
        page += 1  # Increment the page counter

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        jobs = soup.findAll('div', class_='job_seen_beacon')

        if not jobs:
            print("No more jobs found. Ending search.")
            break  # Break the loop if no jobs are found

        for job in jobs:
            data = {}

            job_link_element = job.find('a', href=True)
            job_link = 'https://www.indeed.com' + job_link_element['href'] if job_link_element else "Not listed"
            data['Apply_Link'] = job_link

            # Visit the job page to extract detailed description
            if job_link != "Not listed":
                driver.get(job_link)
                try:
                    link = driver.current_url
                    if "?jk=" in link:
                        try:
                            job_id = link.split('?jk=')[1].split('&')[0]
                        except IndexError:
                            job_id = "Error extracting ID"
                    else:
                        print('link', link)
                        job_id = "Not listed"

                    if job_id is None:
                        data['Job_ID'] = job_id
                        print(job_id)

                        try:
                            title = job.find('h2', class_='jobTitle').text.strip()
                        except AttributeError:
                            title = "Not listed"
                        data['Title'] = title

                        try:
                            company = job.find('span', class_="css-92r8pb").text.strip()
                        except AttributeError:
                            company = "Not listed"
                        data['Company'] = company

                        try:
                            location = job.find('div', class_='css-1p0sjhy').text.strip()
                        except AttributeError:
                            location = "Not listed"
                        data['Location'] = location

                        salary = job.find('div', class_='css-1cvo3fd')
                        data['Salary'] = salary.text.strip() if salary else "Not listed"

                        try:
                            detailed_description = job.find('div', class_='summary').text.strip()
                        except AttributeError:
                            detailed_description = "Not listed"
                        data['Description'] = detailed_description

                    else:
                        print("Skip job: ", job_id)

                except (TimeoutException, NoSuchElementException):
                    detailed_description = "Not listed"
                    data['Description'] = detailed_description

                jobs_data.append(data)

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "fuckthis.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")


Fetching: https://www.indeed.com/jobs?q=environmental+science&l=USA&from=searchOnHP&vjk=3242f2585ff64d120
Page 1 done.
Skip job:  100cf9ab0d80fbfd
Skip job:  d9b504095622a5d2
Skip job:  69b197e5362f788c
Skip job:  10914963c7691226
Skip job:  15b03c439db0492f
Skip job:  2e1cb9dff67c1180
Skip job:  2db42389ad4c0198
link https://www.indeed.com/viewjob?cmp=Insightful-Tutors&t=Test+Preparation+Tutor&jk=9fbde373fa3613d2&xpse=SoAg67I3CPgXBMSdbh0LbzkdCdPP&xkcb=SoCt67M3CPgaVLTaqJ0MbzkdCdPP&vjs=3
Skip job:  Not listed
Skip job:  2582aa991ef46017
link https://www.indeed.com/viewjob?cmp=Forest-Carbon-Works&t=Quality+Assurance+Coordinator&jk=b4868c10f0a71ebf&xpse=SoDm67I3CPgWcJQtmx0LbzkdCdPP&xkcb=SoDw67M3CPgaVLTaqJ0CbzkdCdPP&vjs=3
Skip job:  Not listed
Skip job:  16bcea7a789a883e
Skip job:  d436326cd46ad90c
link https://www.indeed.com/viewjob?cmp=BrightPath-Associates-LLC&t=Environmental+Scientist&jk=f8152a0e7459efec&xpse=SoDh67I3CPgVPcQuyR0LbzkdCdPP&xkcb=SoBX67M3CPgaVLTaqJ0HbzkdCdPP&vjs=3
Skip job

In [37]:
driver = Driver(uc = True)

# Setup explicit wait
wait = WebDriverWait(driver, 20)

In [39]:

# Initialize the list to store job data
jobs_data = []

# Initialize the page variable
page = 0

# Loop through multiple pages
for i in range(0, 200, 10):
    driver.get('https://www.indeed.com/jobs?q=green+economy+entry+level%20&l=united%20states&start=' + str(i))
    driver.implicitly_wait(5)

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            location = job.find('div', class_='css-1p0sjhy').text.strip()
        except AttributeError:
            location = "Not listed"
        data['Location'] = location

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)

    page += 1
    print(f"Page {page} completed.")

# Quit the WebDriver
driver.quit()

# Create DataFrame and save to CSV
df = pd.DataFrame(jobs_data)
df.to_csv('green economy.csv', index=False)
print("Job extraction complete. Data saved to 'sustainability_intern.csv'.")

Page 1 completed.
Page 2 completed.
Page 3 completed.
Page 4 completed.
Page 5 completed.
Page 6 completed.
Page 7 completed.
Page 8 completed.
Page 9 completed.
Page 10 completed.
Page 11 completed.
Page 12 completed.
Page 13 completed.
Page 14 completed.
Page 15 completed.
Page 16 completed.
Page 17 completed.
Page 18 completed.
Page 19 completed.
Page 20 completed.
Job extraction complete. Data saved to 'sustainability_intern.csv'.


In [132]:
jobs_data = []
#https://www.indeed.com/jobs?q=sustainability+entry+level&l=united+states&vjk=e7d490779f2eb29f

for i in range(0,2000,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability+entry+level%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"

          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"
    

        try:
          
            location = job.find('div', class_='css-1p0sjhy').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"
        data['Location'] = location

        '''try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"
'''

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"


        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description


        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('entry_level_scrapping.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 13630 completed.
Page 13645 completed.
Page 13660 completed.
Page 13675 completed.
Page 13690 completed.
Page 13705 completed.
Page 13720 completed.
Page 13735 completed.
Page 13750 completed.
Page 13765 completed.
Page 13780 completed.
Page 13795 completed.
Page 13810 completed.
Page 13825 completed.
Page 13840 completed.
Page 13855 completed.
Page 13870 completed.
Page 13885 completed.
Page 13900 completed.
Page 13915 completed.
Page 13930 completed.
Page 13945 completed.
Page 13960 completed.
Page 13975 completed.
Page 13990 completed.
Page 14005 completed.
Page 14020 completed.
Page 14035 completed.
Page 14050 completed.
Page 14065 completed.
Page 14080 completed.
Page 14095 completed.
Page 14110 completed.
Page 14125 completed.
Page 14140 completed.
Page 14155 completed.
Page 14170 completed.
Page 14185 completed.
Page 14200 completed.
Page 14215 completed.
Page 14230 completed.
Page 14245 completed.
Page 14260 completed.
Page 14275 completed.
Page 14290 completed.
Page 14305

In [136]:
jobs_data = []
#https://www.indeed.com/jobs?q=sustainability+entry+level&l=united+states&vjk=e7d490779f2eb29f

for i in range(0,2000,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability+entry+level%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"

          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"
    

        try:
          
            location = job.find('div', class_='css-1p0sjhy').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"
        data['Location'] = location

        '''try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"
'''

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"


        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description


        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('entry_level_scrapping_second.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 17140 completed.
Page 17155 completed.
Page 17170 completed.
Page 17185 completed.
Page 17200 completed.
Page 17215 completed.
Page 17230 completed.
Page 17245 completed.
Page 17260 completed.
Page 17275 completed.
Page 17290 completed.
Page 17305 completed.
Page 17320 completed.
Page 17335 completed.
Page 17350 completed.
Page 17365 completed.
Page 17380 completed.
Page 17395 completed.
Page 17410 completed.
Page 17425 completed.
Page 17440 completed.
Page 17455 completed.
Page 17470 completed.
Page 17485 completed.
Page 17500 completed.
Page 17515 completed.
Page 17530 completed.
Page 17545 completed.
Page 17560 completed.
Page 17575 completed.
Page 17590 completed.
Page 17605 completed.
Page 17620 completed.
Page 17635 completed.
Page 17650 completed.
Page 17665 completed.
Page 17680 completed.
Page 17695 completed.
Page 17710 completed.
Page 17725 completed.
Page 17740 completed.
Page 17755 completed.
Page 17770 completed.
Page 17785 completed.
Page 17800 completed.
Page 17815

In [114]:
jobs_data = []


for i in range(0,2000,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"
          
          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"


        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description

        
        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('clean_scrapped.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 10355 completed.
Page 10370 completed.
Page 10385 completed.
Page 10400 completed.
Page 10415 completed.
Page 10430 completed.
Page 10445 completed.
Page 10460 completed.
Page 10475 completed.
Page 10490 completed.
Page 10505 completed.
Page 10520 completed.
Page 10535 completed.
Page 10550 completed.
Page 10565 completed.
Page 10580 completed.
Page 10595 completed.
Page 10610 completed.
Page 10625 completed.
Page 10640 completed.
Page 10655 completed.
Page 10670 completed.
Page 10685 completed.
Page 10700 completed.
Page 10715 completed.
Page 10730 completed.
Page 10745 completed.
Page 10760 completed.
Page 10775 completed.
Page 10790 completed.
Page 10805 completed.
Page 10820 completed.
Page 10835 completed.
Page 10850 completed.
Page 10865 completed.
Page 10880 completed.
Page 10895 completed.
Page 10910 completed.
Page 10925 completed.
Page 10940 completed.
Page 10955 completed.
Page 10970 completed.
Page 10985 completed.
Page 11000 completed.
Page 11015 completed.
Page 11030

In [94]:
jobs_data = []


for i in range(0,50,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)


    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"
          
          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"


        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description

        
        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('tryout.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 3550 completed.
Page 3565 completed.
Page 3580 completed.
Page 3595 completed.
Page 3610 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [92]:
jobs_data = []


for i in range(0,50,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)


    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"
          
          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"


        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        
        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('tryout.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 3475 completed.
Page 3490 completed.
Page 3505 completed.
Page 3520 completed.
Page 3535 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [90]:
jobs_data = []


for i in range(0,50,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)


    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}


        try:
          wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
          detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
          detailed_description = "Not listed"
          
          data['Description'] = detailed_description

        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='summary').text.strip()
        except AttributeError:
            detailed_description = "Not listed"
        data['Description'] =jobDescriptionText
        
        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('tryout.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 3400 completed.
Page 3415 completed.
Page 3430 completed.
Page 3445 completed.
Page 3460 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [80]:
jobs_data = []


for i in range(0,2000,10):
    driver.get('https://www.indeed.com/jobs?q=sustainability%20&l=united%20states&start='+str(i))
    driver.implicitly_wait(5)


    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs_final.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 256 completed.
Page 271 completed.
Page 286 completed.
Page 301 completed.
Page 316 completed.
Page 331 completed.
Page 346 completed.
Page 361 completed.
Page 376 completed.
Page 391 completed.
Page 406 completed.
Page 421 completed.
Page 436 completed.
Page 451 completed.
Page 466 completed.
Page 481 completed.
Page 496 completed.
Page 511 completed.
Page 526 completed.
Page 541 completed.
Page 556 completed.
Page 571 completed.
Page 586 completed.
Page 601 completed.
Page 616 completed.
Page 631 completed.
Page 646 completed.
Page 661 completed.
Page 676 completed.
Page 691 completed.
Page 706 completed.
Page 721 completed.
Page 736 completed.
Page 751 completed.
Page 766 completed.
Page 781 completed.
Page 796 completed.
Page 811 completed.
Page 826 completed.
Page 841 completed.
Page 856 completed.
Page 871 completed.
Page 886 completed.
Page 901 completed.
Page 916 completed.
Page 931 completed.
Page 946 completed.
Page 961 completed.
Page 976 completed.
Page 991 completed.


In [73]:


jobs_data = []
max_pages = 5  # Set a maximum number of pages to scrape
page = 0

while page < max_pages:
    url = f"https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk={page * 10}"
    driver.get(url)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")

# Check for the 'Next' button - this may vary depending on Indeed's page structure
next_button = soup.find('a', {'aria-label': 'Next'})
if next_button and 'href' in next_button.attrs:
  # Indeed uses a 'start' parameter to paginate
  params['start'] += 10


driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs_one.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 16 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [64]:


jobs_data = []
max_pages = 5  # Set a maximum number of pages to scrape
page = 0

while page < max_pages:
    url = f"https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk={page * 10}"
    driver.get(url)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")
    
page += 1

driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs_one.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 16 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [62]:


jobs_data = []
max_pages = 5  # Set a maximum number of pages to scrape
page = 0

while page < max_pages:
    url = f"https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk={page * 10}"
    driver.get(url)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)
        page += 1
    print(f"Page {page + 1} completed.")
    

driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 16 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [42]:


jobs_data = []
max_pages = 5  # Set a maximum number of pages to scrape
page = 0

while page < max_pages:
    url = f"https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk={page * 10}"
    driver.get(url)

    #https://www.indeed.com/jobs?q=sustainability++jobs&l=USA&vjk=0cdb55029eb47b25

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)

    print(f"Page {page + 1} completed.")
    page += 1

driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

Page 1 completed.
Page 2 completed.
Page 3 completed.
Page 4 completed.
Page 5 completed.
Job extraction complete. Data saved to 'indeed_jobs.csv'.


In [13]:

# Initialize jobs_data list
jobs_data = []

# Set maximum number of pages
max_pages = 3

# Loop through each page until max_pages is reached
page = 0
while page < max_pages:
    page_val = page * 12  # Pagination step
    url = f"https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e={page_val}"
    print("Fetching:", url)
    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    # Extract job listings
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found. Ending search.")
        break

    # Loop through each job listing
    for job in jobs:
        data = {}
        data['Apply_Link'] = 'Not listed'

        try:
            title = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            title = "Not listed"
        data['Title'] = title

        try:
            company = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            company = "Not listed"
        data['Company'] = company

        try:
            location = job.find('div', class_='css-1p0sjhy').text.strip()
        except AttributeError:
            location = "Not listed"
        data['Location'] = location

        salary = job.find('div', class_='css-1cvo3fd')
        data['Salary'] = salary.text.strip() if salary else "Not listed"

        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description
        jobs_data.append(data)

    print(f"Page {page + 1} done.")
    page += 1  # Move to the next page

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "242024.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")

Fetching: https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e=0
Page 1 done.
Fetching: https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e=12
Page 2 done.
Fetching: https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e=24
Page 3 done.
Data saved to CSV file successfully.


In [22]:

# Initialize jobs_data list
jobs_data = []

# Set maximum number of pages
max_pages = 3

# Loop through each page until max_pages is reached
page = 0
while page < max_pages:
    page_val = page * 12  # Pagination step
    url = f"https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e={page_val}"
    print("Fetching:", url)
    driver.get(url)

    # Wait for the job listings to be loaded
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    # Extract job listings
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found. Ending search.")
        break

    # Loop through each job listing
    for job in jobs:
        data = {}
        data['Apply_Link'] = 'Not listed'

        try:
            title = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            title = "Not listed"
        data['Title'] = title

        try:
            company = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            company = "Not listed"
        data['Company'] = company

        try:
            location = job.find('div', class_='css-1p0sjhy').text.strip()
        except AttributeError:
            location = "Not listed"
        data['Location'] = location

        salary = job.find('div', class_='css-1cvo3fd')
        data['Salary'] = salary.text.strip() if salary else "Not listed"

        try:
            wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
            detailed_description = driver.find_element(By.ID, 'jobDescriptionText').text.strip()
        except (TimeoutException, NoSuchElementException):
            detailed_description = "Not listed"

        data['Description'] = detailed_description
        jobs_data.append(data)

    print(f"Page {page + 1} done.")
    page += 1  # Move to the next page

# Close the WebDriver after finishing
driver.quit()

# Save data to CSV file
csv_file_path = "242024.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ['Title', 'Company', 'Location', 'Salary', 'Description', 'Apply_Link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for job_data in jobs_data:
        writer.writerow(job_data)

print("Data saved to CSV file successfully.")

Fetching: https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e=0
Page 1 done.
Fetching: https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e=12
Page 2 done.
Fetching: https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e=24
Page 3 done.
Data saved to CSV file successfully.
